This notebook replicates the brain-related analyses. Note: the fMRI data can be downloaded into the data folder from here: https://dataspace.princeton.edu/jspui/handle/88435/dsp01nz8062179

## Import libraries

In [ ]:
import sys
import numpy as np
from os.path import join as opj
from nilearn import datasets, surface
from nilearn.image import concat_imgs, load_img, new_img_like
from scipy.stats import ttest_1samp as ttest

## Import analysis helpers

In [ ]:
sys.path.insert(0, abspath('../../helpers/'))
from analysis_helpers import r2z

## Set paths & parameters

In [ ]:
datadir = '../../../data/processed/'

## Load in permutations

In [ ]:
video_perms = [load_img(opj(datadir, f'searchlight_video/perm{perm}.nii.gz')) for perm in range(100)]
video_perms = concat_imgs(video_perms).dataobj.astype(np.float64)

recall_perms = [load_img(opj(datadir, f'searchlight_recall/perm{perm}.nii.gz')) for perm in range(100)]
recall_perms = concat_imgs(recall_perms).dataobj.astype(np.float64)

## Load in real data

In [ ]:
ref_img = load_img(opj(datadir, 'searchlight_video/ref_img.nii.gz'))
                   
subs = range(1, 18)
vid_imgs = []
rec_imgs = []
for sub in subs:
    sub_vdata = np.load(opj(datadir, f'searchlight_video/sub{sub}.npy'), allow_pickle=True)
    sub_rdata = np.load(opj(datadir, f'searchlight_recall/sub{sub}.npy'), allow_pickle=True)
    vid_img = new_img_like(ref_img, sub_vdata.astype(np.float64))
    rec_img = new_img_like(ref_img, sub_rdata.astype(np.float64))
    vid_imgs.append(vid_img)
    rec_imgs.append(rec_img)
    
vid_imgs = concat_imgs(vid_imgs)
rec_imgs = concat_imgs(rec_imgs)

## Get stats for real data

In [ ]:
video_data = vid_imgs.dataobj.astype(np.float64)
video_statmap = ttest(np.moveaxis(r2z(video_data), -1, 0), 0).statistic
video_img = new_img_like(ref_img, video_statmap.astype(np.float64))


recall_data = rec_imgs.dataobj.astype(np.float64)
recall_statmap = ttest(np.moveaxis(r2z(recall_data), -1, 0), 0).statistic
recall_img = new_img_like(ref_img, recall_statmap.astype(np.float64))

## Do permutation correction

In [ ]:
real_video = video_img.dataobj.astype(np.float64)
real_recall = recall_img.dataobj.astype(np.float64)

zval_video = (real_video - np.nanmean(video_perms, axis=3)) / np.nanstd(video_perms, axis=3)
pval_video = (real_video[:, :, :, np.newaxis] < video_perms).sum(axis=3) / 100
zval_recall = (real_recall - np.nanmean(recall_perms, axis=3)) / np.nanstd(recall_perms, axis=3)
pval_recall = (real_recall[:, :, :, np.newaxis] < recall_perms).sum(axis=3) / 100

zval_video = np.nan_to_num(zval_video)
pval_video = np.nan_to_num(pval_video)
zval_recall = np.nan_to_num(zval_recall)
pval_recall = np.nan_to_num(pval_recall)

## Export p-value maps for neurosynth decoding

In [ ]:
ns_vid_map = np.copy(pval_video)
ns_rec_map = np.copy(pval_recall)

# isolate voxels more highly correlated with real trajectories than permuted ones
ns_vid_map[zval_video <= 0] = 0
ns_rec_map[zval_recall <= 0] = 0

ns_vid_map = new_img_like(ref_img, ns_vid_map.astype(np.float64))
ns_rec_map = new_img_like(ref_img, ns_rec_map.astype(np.float64))

ns_vid_map.to_filename(opj(datadir, 'searchlight_video', 'ns_map_video.nii.gz'))
ns_rec_map.to_filename(opj(datadir, 'searchlight_recall', 'ns_map_recall.nii.gz'))

## Threshold

In [ ]:
zval_video[pval_video > .05] = 0
zval_video[zval_video < 0] = 0

zval_recall[pval_recall > .05] = 0
zval_recall[zval_recall < 0] = 0

zmap_video = new_img_like(ref_img, zval_video.astype(np.float64))
zmap_recall = new_img_like(ref_img, zval_recall.astype(np.float64))

## Convert to surface maps

In [ ]:
fsaverage = datasets.fetch_surf_fsaverage(mesh='fsaverage5')
vid_texture_pl = surface.vol_to_surf(zmap_video, fsaverage.pial_left)
vid_texture_pr = surface.vol_to_surf(zmap_video, fsaverage.pial_right)
rec_texture_pl = surface.vol_to_surf(zmap_recall, fsaverage.pial_left)
rec_texture_pr = surface.vol_to_surf(zmap_recall, fsaverage.pial_right)

## Export for plotting

In [ ]:
np.save(opj(datadir, 'searchlight_video', 'video_surface_left.npy'), vid_texture_pl)
np.save(opj(datadir, 'searchlight_video', 'video_surface_right.npy'), vid_texture_pr)
np.save(opj(datadir, 'searchlight_recall', 'recall_surface_left.npy'), rec_texture_pl)
np.save(opj(datadir, 'searchlight_recall', 'recall_surface_right.npy'), rec_texture_pr)